In [9]:
# diff of llm and chatAi
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# text-davinchi-003 is stopped
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")
# model_name = "gpt-3.5-turbo"
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a,b

('\n\nAs of 2021, there are eight official planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Pluto was previously considered a planet, but it was reclassified as a dwarf planet in 2006. There may be other planets outside of our solar system, but they have not been officially confirmed.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

# chatAi options
chat = ChatOpenAI(
    temperature=0.1
)

# hard code message
#messages = [
#   SystemMessage(content= "You are a geography expert. And you only reply in korean"),
#   AIMessage(content= "안녕하세요 내 친구여"),
#   HumanMessage(content= "what is the distance between Korea and Italy. Also what is your name?")
#]

#chat.predict_messages(messages)

messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "안녕하세요 내 친구여 저는 {name}이라 합니다"),
        ("human", "what is the distance between {country_a} and {country_b}. Also what is your name?")
    ]
)

#prompt = template.format(country_a = "Japan", country_b = "America")
#chat.predict(prompt)

prompt = messages.format_messages(
    language = "Korean",
    name = "Jung pal",
    country_a = "Korea",
    country_b = "Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='한국과 태국 사이의 거리는 약 3,300km입니다. 제 이름은 정팔입니다. 어떻게 도와드릴까요?')

In [23]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

#p.parse("Hello, how, are, you")

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. And do not reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets?"
)

result = chat.predict_messages(prompt)

p.parse(result.content)

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [24]:
# chain all upper operation
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pocket monsters"
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']